<a href="https://colab.research.google.com/github/LeeTunMT/python/blob/main/QHTT_MT_TP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 46.5 MB/s eta 0:00:00


In [3]:
from pulp import *
solver = PULP_CBC_CMD()

In [13]:
!pip install --upgrade pulp
from pulp import *
solver = PULP_CBC_CMD()
from scipy.optimize import linprog
import numpy as np

def input_problem():
    n = int(input("Enter the number of variables: "))

    variables = {}
    for i in range(n):
        var_name = input(f"Enter the name of variable {i+1}: ")
        variables[var_name] = LpVariable(var_name, lowBound=0)

    obj = input("Enter the objective function (e.g., 3*x + 5*y): ")
    obj_type = input("Objective type (min/max): ")

    m = int(input("Enter the number of constraints: "))
    constraints = []
    for i in range(m):
        cons = input(f"Enter constraint {i+1} (e.g., x + y <= 100): ")
        constraints.append(cons)

    return n, variables, obj, obj_type, m, constraints

def convert_to_standard_form(n, variables, obj, obj_type, m, constraints):
    # Convert constraints to standard form (add slack variables)
    standard_constraints = []
    for cons in constraints:
        standard_constraints.append(cons + " <= 0")

    # Create initial tableau for phase one
    tableau = np.zeros((m + 1, n + m + 1))
    for i in range(m):
        tableau[i, :n] = eval(constraints[i], {**variables, **locals()})
        tableau[i, n + i] = 1
        tableau[i, -1] = eval(constraints[i].split()[1])

    # Set up the objective function for phase one
    c_phase_one = np.zeros(n + m + 1)
    c_phase_one[-1] = -1  # Minimize the artificial variable

    return tableau, c_phase_one

def solve_lp(n, variables, obj, obj_type, m, constraints, method):
    # Convert constraints to standard form (add slack variables)
    standard_constraints = []
    for cons in constraints:
        standard_constraints.append(cons + " <= 0")

    # Solve using the chosen method
    if method == "geometric":
        if obj_type == "max":
            result = linprog(c=[-1, -1], A_ub=standard_constraints, bounds=[(0, None), (0, None)])
        else:
            result = linprog(c=[1, 1], A_ub=standard_constraints, bounds=[(0, None), (0, None)])
        return result
    elif method == "simplex":
        # Implement simplex method here
      def objective_function(coefficients, variables):
          return sum(c * x for c, x in zip(coefficients, variables))

      # Hàm kiểm tra điều kiện kết thúc
      def is_optimal(coefficients):=
          return all(c >= 0 for c in coefficients)

      # Triển khai phương pháp đơn hình
      def simplex_method(coefficients, constraints):
          # Khởi tạo biến cơ sở và biến không cơ sở
          # Tạo bảng đơn hình ban đầu
          num_iterations = 0

          while not is_optimal(coefficients):
              # Chọn biến ra và biến vào
              # Thực hiện phép xoay từ vựng
              # Cập nhật bảng đơn hình
              num_iterations += 1

          return coefficients, num_iterations  # Kết quả tối ưu và số bước lặp

    elif method == "two_phase":
      def solve_two_phase_simplex(tableau, c_phase_one):
          while True:
              # Check if the current solution is optimal
              if np.all(tableau[-1, :-1] >= 0):
                  break

              # Find the pivot column (entering variable)
              pivot_col = np.argmin(tableau[-1, :-1])

              # Find the pivot row (leaving variable)
              ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
              ratios[ratios < 0] = np.inf  # Ignore negative ratios
              pivot_row = np.argmin(ratios)

              # Perform pivot operation
              pivot_value = tableau[pivot_row, pivot_col]
              tableau[pivot_row, :] /= pivot_value
              for i in range(m + 1):
                  if i != pivot_row:
                      factor = tableau[i, pivot_col]
                      tableau[i, :] -= factor * tableau[pivot_row, :]

          return tableau
    elif method == "dual":
      def solve_dual_simplex(tableau, c_phase_one):
          while True:
              # Check if the current solution is optimal
              if np.all(tableau[-1, :-1] >= 0):
                  break

              # Find the pivot column (entering variable)
              pivot_col = np.argmin(tableau[-1, :-1])

              # Find the pivot row (leaving variable)
              ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
              ratios[ratios < 0] = np.inf  # Ignore negative ratios
              pivot_row = np.argmin(ratios)

              # Perform pivot operation
              pivot_value = tableau[pivot_row, pivot_col]
              tableau[pivot_row, :] /= pivot_value
              for i in range(m + 1):
                  if i != pivot_row:
                      factor = tableau[i, pivot_col]
                      tableau[i, :] -= factor * tableau[pivot_row, :]

          return tableau
    else:
        print("Invalid method choice.")
        return None

def print_results(result, variables):
    if result is not None and result.success:
        print("Optimal solution:")
        for var, val in zip(variables.keys(), result.x):
            print(f"{var} = {round(val, 4)}")
        print("Optimal value:", round(result.fun, 4))
        if result.status == 0:
            print("Unique solution.")
        elif result.status == 2:
            print("Infeasible problem.")
        elif result.status == 3:
            print("Unbounded problem.")
        else:
            print("Other cases.")
    else:
        print("The problem is infeasible or unbounded.")

if __name__ == "__main__":
    n, variables, obj, obj_type, m, constraints = input_problem()
    method = input("Choose a solution method (geometric/simplex/two_phase/dual): ")
    result = solve_lp(n, variables, obj, obj_type, m, constraints, method)
    print_results(result, variables)


Enter the number of variables: 2
Enter the name of variable 1: x
Enter the name of variable 2: y
Enter the objective function (e.g., 3*x + 5*y): x + y
Objective type (min/max): max 
Enter the number of constraints: 5
Enter constraint 1 (e.g., x + y <= 100): x + 2*y <= 6
Enter constraint 2 (e.g., x + y <= 100): 2*x + y <= 8
Enter constraint 3 (e.g., x + y <= 100): y <= 2
Enter constraint 4 (e.g., x + y <= 100): x >= 0
Enter constraint 5 (e.g., x + y <= 100): y >= 0
Choose a solution method (geometric/simplex/two_phase/dual): simplex
The problem is infeasible or unbounded.
